In [1]:
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
import matplotlib.pyplot as plt
import numpy as np

def gen_matrix(image, nb_channels, random_state):
      matrix_A = np.array([[0, -1, 0],
                           [-1, 4, -1],
                           [0, -1, 0]])
      matrix_B = np.array([[0, 0, 0],
                          [0, -4, 1],
                          [0, 2, 1]])
      if random_state.rand() < 0.5:
          return [matrix_A] * nb_channels
      else:
          return [matrix_B] * nb_channels
# aug = iaa.Convolve(matrix=gen_matrix)

seq = iaa.Sequential([
    ## unsure if Crop is likely to be useful offsett may be better
#     iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    ## definitely worth doing Fliplr as it doubles the size.
#     iaa.Fliplr(0.5), # horizontally flip 50% of the images
    ## GaussianBlur is also worth performing on for generalising
    ## possibly ignore it for my own data?  
#     iaa.GaussianBlur(sigma=(0, 3.0)), # blur images with a sigma of 0 to 3.0
    ## Noise is also worth doing for generalising.
#     iaa.AdditiveGaussianNoise(scale=0.1*255) #looks like static
#     iaa.EdgeDetect(alpha=(0.0, 1.0))
    
    ## I think convolve is probably useful if you define it with the sharpness to demonstrate
    ## the advantages of using Convolve is to view what the Computer might see.
    iaa.Convolve(matrix=gen_matrix),
    ## sharpeness definitely feels useful for defining the edges of the area.
#     iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0))
    ## probably worth doing sometimes, it might be better with 
#     iaa.ContrastNormalization((0.5, 1.5))
#     iaa.ContrastNormalization((0.5, 1.5), per_channel=0.5),
    ## Affine scale
    ## might be an issue due to where the signs are typically located don't increase the sign beyond 1.0
    ## unless you write a function to discard bounding boxes outside the actual area.
#     iaa.Affine(scale=(0.35, .90)),
    ## scale indepently on each axis indepently changing axis, might be really useful
#     iaa.Affine(scale={'x': (0.5, 1), 'y': (0.35, 1.0)}),
    ## this could be useful however, it would require it the translation to not cut off the signs in the images
    ## possible function to ensure it's not missing
    ## note though it's on percent
#     iaa.Affine(translate_percent={'x': (-0.02, 0.2), 'y': (-0.2, 0.2)})
    ## note the below is a pixel translation which might actually be better due to it being more minor
#     iaa.Affine(translate_pixel={'x': (-20, 20), 'y': (-20, 20)})
    ## note that this will be worth while if it's using with scaling and/or doesn't cut off
    ## the actual sign
#     iaa.Affine(rotate=(-45, 45))
    ## shear is definitely worth including in the data augmentation process
#     iaa.Affine(shear=(-160,160))
    ## unsure of what the mode kwarg is doing currently
#     iaa.Affine(translate_percent={"x": (-0.20)}, mode=ia.ALL, cval=(0, 255))
    ## Really useful on low level distortion
#     iaa.PiecewiseAffine(scale=(0.008, 0.0095))
    ## use ElasticTransformation to add random can pass in tuples for randomness
    ## to the distortion field
    iaa.ElasticTransformation(alpha=(0, 5.0), sigma=(0.25, 1))

])
img1 = cv2.imread('/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_235.jpg')

img = cv2.imread('/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_105.jpg')
# cv2.imshow('test', img)
imgs = [img, img1]
print(type(img))
# seq.show_grid(imgs[0], cols=8, rows=8)
# cropped = img[7:100, 22:67]
# # cv2.imshow('cropped', cropped)
# # cv2.waitKey(0)

img_aug = seq.augment_image(img)
# seq.show_grid(img_aug, cols=8, rows=8)
# print(img.shape)
# print(img_aug.shape)
# print(len(img_aug))

# cv2.imshow('aug??', img_aug)
# cv2.waitKey(0)

# # for batch_idx in range(1000):
# #     # 'images' should be either a 4D numpy array of shape (N, height, width, channels)
# #     # or a list of 3D numpy arrays, each having shape (height, width, channels).
# #     # Grayscale images must have shape (height, width, 1) each.
# #     # All images must have numpy's dtype uint8. Values are expected to be in
# #     # range 0-255.
# #     images = load_batch(batch_idx)
# #     images_aug = seq.augment_images(images)
# #     train_on_images(images_aug)

<class 'numpy.ndarray'>


# Write out one image as new.

1. It needs the path of the image.
2. The bounding boxes of the image.

In [2]:
import os
import xml.etree.ElementTree as ET
import cv2

base_dir = '/Users/datascience4/Documents/training3'
print(len(os.listdir(base_dir)))
xml_files = os.listdir(base_dir)
xml_files = [x for x in os.listdir(base_dir) if '.xml' in x]
print(xml_files)
items = []

## come back to this

#let's read the first xml
test_xml = os.path.join(base_dir, xml_files[0])
tree = ET.parse(test_xml)
root = tree.getroot()
## path can be used for cv2 read image
path = root.find('path').text
print(path)
## find the object and xmin, ymin, xmax, ymax
for child in root.iter('object'):
    if child.find('bndbox'):
        bndbox = child.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        
print(xmin, ymin, xmax, ymax)

ia.seed(1)
## pass the path into cv2
image = cv2.imread(path)
# cv2.imshow('path', image)
# cv2.waitKey(0)

## bounding boxes now for the image takes a list
bbs = ia.BoundingBoxesOnImage([
    ia.BoundingBox(x1=xmin, y1=ymin, x2=xmax, y2=ymax)
], shape=image.shape)

seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5)), # change brightness, doesn't affect bb
    iaa.Affine(
        translate_px={"x": 40, "y": 60},
        scale=(0.5, 0.7)
    ) # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
])
## make our sequence deterministic.
# we can now apply it to the image and then to the bbs and it will
# lead to the same augmentations
# IMPORTANT: call this once PER BATCH, otherwise you will always
# get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##Augment bbs and images.
## as we only have one image and list of bbs we use
## [image] and [bbs] to turn both into lists (batches) for the
## functions and then [0] to reverse that. In a real experiment, your
## variables would likely already be lists.
image_aug = seq_det.augment_images([image])[0]
bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

##print the coordinates before/after augmentation(see below)
## use .x1_int, y1_int, ... to get interger coordinates
for i in range(len(bbs.bounding_boxes)):
    before = bbs.bounding_boxes[i]
    after = bbs_aug.bounding_boxes[i]
    print(f"BB {i}:, ({before.x1, before.y1, before.x2, before.y2} \
          -> ({after.x1, after.y1, after.x2, after.y2})")
    
# image with bbs before/after augmentation (shown below)
image_before = bbs.draw_on_image(image, thickness=2)
cv2.imshow('image before', image_before)
cv2.waitKey(0)

image_after = bbs_aug.draw_on_image(image_aug, thickness=2, color=[0, 0, 255])
cv2.imshow('image after', image_after)
cv2.waitKey(0)

385
['A3400_NB1_RAV_R07_180516084515_frame_3598.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2862.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2447.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2335.xml', 'A3400_NB1_RAV_R07_180516084515_frame_4022.xml', 'A3400_NB1_RAV_R07_180516084515_frame_224.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2255.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2527.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3163.xml', 'A3400_NB1_RAV_R07_180516084515_frame_350.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3375.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3374.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3837.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3162.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2240.xml', 'A3400_NB1_RAV_R07_180516084515_frame_225.xml', 'A3400_NB1_RAV_R07_180516084515_frame_4023.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2446.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2863.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3599.xml', 'A3400_NB1_RAV_R07

48

BB 0: (65.0000, 100.0000, 200.0000, 150.0000) -> (458.1977, 376.8130, 537.5725, 406.2110)
BB 1: (150.0000, 80.0000, 200.0000, 130.0000) -> (508.1745, 365.0537, 537.5725, 394.4518)


48